#  Vertex Inference Unified

This notebook uses the unified `google-genai` library (imported as `from google import genai`). It supports:
- **Vertex AI Backend:** Uploads videos to GCS during the 'Prepare' step.
- **Gemini API Backend:** Uploads videos using the **File API** during the 'Prepare' step.

**Pipeline Steps:**
1.  **Import Libraries & Configure.**
2.  **Config** - Set up the configuration for the pipeline, including the model, model config, prompts, and prompt config.
2.  **Initialize Clients:** Set up AI client and Storage client.
3.  **(Only the first time) Fetch Dataset:** Downloads metadata from HuggingFace.
4.  **(Only the first time on each API type) Download, Extract & Prepare Videos:** Downloads, extracts, uploads (GCS/File API). Updates metadata.
5.  **Bulk Inference (Async):** Performs inference using pre-uploaded video resources.
6.  **Single Prompt Testing (UI):** Allows interactive testing of the video with prompts

### Notes:

1. After switching from Vertex to Gemini and vice versa, be sure to follow the steps:
    - Run all cells in order to re-upload the videos to the correct storage client, you can enable the SKIP_DOWNLOAD and SKIP_EXTRACT flags to skip the download and extraction steps. Only the upload step is needed

2. Gemini API's file client has a expiry time of 1 day or so for the uploaded files. You may need to follow the steps above to re-upload the files.

## Import Libraries

In [ ]:
# Cell 1: Imports (Corrected for `google.genai`)
import os
import csv
import json
import logging
import time
import random
import requests
import datetime
import zipfile
import math
import sys
import asyncio
from typing import Dict, List, Optional, Set, Tuple, Any
from collections import defaultdict
from pathlib import Path
import shutil
import subprocess
import tempfile
import fractions

# Google Cloud & AI Libraries (Unified SDK)
try:
    import google.genai as genai
    from google.genai import types
    from google.genai import errors as genai_errors
    from google.api_core import exceptions as api_core_exceptions
    # GCS Client (Optional, for Vertex Mode)
    try:
        from google.cloud import storage
        GCS_AVAILABLE = True
    except ImportError:
        print("INFO: google-cloud-storage not found. Vertex AI GCS operations unavailable.")
        storage = None
        GCS_AVAILABLE = False
    print("`google.genai` SDK and helpers imported successfully.")
except ImportError as e:
     print(f"ERROR: Failed to import Google libraries: {e}. Install: pip install google-genai google-api-core google-cloud-storage")
     genai = None; types = None; genai_errors = None; api_core_exceptions = None
     storage = None; GCS_AVAILABLE = False
     raise ImportError("FATAL: `google.genai` or `google-api-core` SDK not found.")

# Data Handling & Progress
from datasets import load_dataset
import pandas as pd
from tqdm.notebook import tqdm

# UI Elements
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output

# Async in Notebook
import nest_asyncio
nest_asyncio.apply()

## Config Settings

In [ ]:
# --- GCP Configuration ---

PROJECT_ID = "YOUR_PROJECT_ID_HERE" # Your Google Cloud Project ID (Needed for GCS and Vertex AI mode)
LOCATION = "us-central1"      # Your Google Cloud Region (Needed for Vertex AI mode)
GCS_BUCKET = "YOUR_GCS_BUCKET_HERE" # Your GCS bucket name (Needed for video storage)

# --- Choose Backend Mode ---
# Set USE_VERTEX to True to use the Vertex AI backend (requires ADC or service account auth).
# Set USE_VERTEX to False to use the Gemini API backend (requires GEMINI_API_KEY).
USE_VERTEX = False  # <-- CHANGE THIS TO True TO USE VERTEX AI

# --- Gemini API Key (Only required if USE_VERTEX is False) ---
# IMPORTANT: Replace with your actual Gemini API Key if USE_VERTEX is False.
# Consider loading from environment variables (GOOGLE_API_KEY) or a secure secrets manager.
GEMINI_API_KEY = ""  # Replace with your actual Gemini API Key


# --- File Paths ---
DATASET_CSV = "dataset.csv"               # Input dataset metadata from HuggingFace
METADATA_FILE = "video_metadata_vertex.csv" if USE_VERTEX else "video_metadata_non_vertex.csv"      # Stores video info: video_id, local_path, gcs_uri (if Vertex), question data
RESULTS_FILE = "results_ccot_full_inference.csv"              # Output file for inference predictions
DOWNLOADS_DIR = "downloads"               # Directory for downloaded zip file
EXTRACTED_VIDEOS_DIR = "extracted_videos" # Directory storing extracted .mp4 files locally
SPEED_VIDEOS_DIR = "speed_videos"         # Stores sped up/slowed down videos
HF_CACHE_DIR = "./hf_cache"               # Cache directory for HuggingFace datasets

# --- Step 1: Fetch Dataset Configuration ---
HF_DATASET_NAME = "lmms-lab/AISG_Challenge" # HuggingFace dataset identifier
HF_DATASET_SPLIT = "test"                 # Dataset split to use
SKIP_FETCH = False                        # Set True to skip fetching if DATASET_CSV exists

# --- Step 2: Download & Prepare Videos Configuration ---
VIDEO_ZIP_URL = "https://huggingface.co/datasets/lmms-lab/AISG_Challenge/resolve/main/Benchmark-AllVideos-HQ-Encoded-challenge.zip?download=true"
ZIP_FILE_NAME = "all_videos.zip"
SKIP_DOWNLOAD_ZIP = True                 # Set True to skip downloading if zip exists
SKIP_EXTRACT = True                   # Set True to skip extraction if videos exist locally
SKIP_PREPARE = True                    # Set True to skip video preparation (GCS upload for Vertex, metadata update)
MAX_VIDEOS_TO_PROCESS = None              # Limit videos for testing (e.g., 5), None for all
UPLOAD_BATCH_SIZE_GCS = 10                # Batch size for GCS uploads (Vertex mode only)

# --- Inference Configuration ---
# Choose a model name compatible with your selected method (Vertex AI or Gemini API)

# Examples:

# Vertex AI: gemini-2.0-flash, gemini-2.0-flash-lite, gemini-2.0-pro-exp-02-05, gemini-2.0-flash-thinking-exp-01-21
# Rate limits: https://cloud.google.com/vertex-ai/generative-ai/docs/quotas#gemini-2.0-flash
# Basically 500 requests per minute for 2.0-flash and 2.0-flash-lite (unlimited), 10 requests per minute for 2.0-pro-exp-02-05, gemini-2.5-flash-preview-04-17

# Gemini API: gemini-2.0-flash, gemini-2.0-flash-lite, gemini-2.0-flash-thinking-exp-01-21, gemini-2.5-pro-exp-03-25
# Rate limits: https://ai.google.dev/gemini-api/docs/rate-limits#tier-1
# For free tier: 30 requests per minute for 2.0-flash and 2.0-flash-lite, 10 requests per minute for 2.0-pro-exp-02-05
# For tier-1: 2000 requests per minute for 2.0-flash and 2.0-flash-lite (have to pay), 10 requests per minute for 2.0-pro-exp-02-05 and gemini-2.0-flash-thinking-exp-01-21, gemini-2.5-flash-preview-04-17

# 1.0=normal speed, 0.5=half speed, etc.
VIDEO_SPEED_FACTOR = 0.5

# --- Setup Derived Paths & Directories ---
zip_file_path = Path(DOWNLOADS_DIR) / ZIP_FILE_NAME
extracted_videos_path = Path(EXTRACTED_VIDEOS_DIR)
speed_videos_path = Path(SPEED_VIDEOS_DIR) / str(VIDEO_SPEED_FACTOR)
Path(DOWNLOADS_DIR).mkdir(parents=True, exist_ok=True)
extracted_videos_path.mkdir(parents=True, exist_ok=True)
speed_videos_path.mkdir(parents=True, exist_ok=True)
Path(HF_CACHE_DIR).mkdir(parents=True, exist_ok=True)

# --- Logging Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler(sys.stdout)])
logger = logging.getLogger(__name__)

# --- Configuration Validation & Display --- #
warnings_found = False
if USE_VERTEX:
    if not PROJECT_ID or PROJECT_ID == "your-gcp-project-id":
        logger.error("Vertex AI mode requires PROJECT_ID to be set.")
        warnings_found = True
    if not LOCATION:
        logger.error("Vertex AI mode requires LOCATION to be set.")
        warnings_found = True
    if not GCS_BUCKET or GCS_BUCKET == "your-gcs-bucket-name":
        logger.error("Vertex AI mode requires GCS_BUCKET for video uploads.")
        warnings_found = True
    if not GCS_AVAILABLE:
        logger.error("Vertex AI mode requires 'google-cloud-storage', but it's not installed.")
        warnings_found = True
else: # Gemini API Mode
    # Check API Key (explicit or env var)
    effective_api_key = GEMINI_API_KEY if GEMINI_API_KEY != "YOUR_API_KEY_HERE" else os.environ.get("GOOGLE_API_KEY")
    if not effective_api_key:
        logger.error("Gemini API mode requires GEMINI_API_KEY or GOOGLE_API_KEY environment variable.")
        warnings_found = True
    else:
        # Don't store the key in the config display if loaded from env
        if GEMINI_API_KEY == "YOUR_API_KEY_HERE" and os.environ.get("GOOGLE_API_KEY"):
            GEMINI_API_KEY = "(Loaded from GOOGLE_API_KEY env var)"
        logger.info("Gemini API mode configured. Videos will be uploaded via File API.")

if warnings_found:
     print("\n\n************************* WARNING *************************")
     print("Configuration errors detected above. Execution might fail.")
     print("***********************************************************\n")

# Main Model Selection (Innovation 1)
Agentic CoT - Chain of Thought + Summary

## 1. Select CoT Model and Summary Model

### Generated Questions Modesl

In [ ]:
QUESTIONS_MODEL_NAME = "gemini-2.0-flash"
QUESTIONS_DIR = os.path.join(f"generated_questions/{QUESTIONS_MODEL_NAME}", "questions.csv")

### CoT output models

In [ ]:
# To See All Available Models, Go into the CoT_ouput_models.py file

from models.CoT_ouput_models import get_cot_model

CoT_model_list = ["gemini-2.0-flash","gemini-2.5-flash-preview-04-17", "gemini-2.5-pro-preview-03-25"]

MODEL_NAME, SYSTEM_PROMPT, PROMPT_TEMPLATES, CONFIG, REQUESTS_PER_MINUTE, MAX_RETRIES, MAX_ASYNC_WORKERS = get_cot_model(CoT_model_list[2])

### Summary Model Selection

In [ ]:
# To see all available models, go into the Summary_models.py file
from models.Summary_models import get_summary_model
summary_model_list = ["gemini-2.0-flash-ver1", "gemini-2.0-flash-ver2", "gemini-2.0-flash-ver3"]

QUESTION_MODEL_NAME, QUESTION_SYSTEM_PROMPT, QUESTION_CONFIG = get_summary_model(summary_model_list[0])

**Summary Model Prompt Versions (Gemini Flash) Comparison**

Concise summary of different post-processing approaches for LLM outputs with `thinking` blocks.


#### Version 1:  No Yap Model (Extractor Model)

*   **Action:** Extracts final answer, minimal changes.
*   **`thinking`:** Mostly ignored; extracts text after it.
*   **Output:** Final answer only (For MCQ, letter only). "No yap".

#### Version 2: Detailed Reformatter & Explainer

*   **Action:** Answer first, then detailed *synthesized* explanation.
*   **`thinking`:** Source for detailed step-by-step explanation; removes examiner notes.
*   **Output:** `Answer \n\n Explanation: \n [Detailed Synth. Explanation]`

#### Version 3: Light Touch Reformatter & Preserver

*   **Action:** Answer first, then *original* `thinking` block.
*   **`thinking`:** Preserved in output (within ``` ```).
*   **Output:** `Answer \n\n ```thinking\n[Original Thinking]\n``` `

# Basic Initialization

## Initialize Google Cloud Clients

In [ ]:
storage_client = None
ai_client = None

# --- Initialize Generative AI Client (`google.genai`) --- #
display(Markdown("### Initializing Generative AI Client (`google.genai`)"))
try:
    if USE_VERTEX:
        display(Markdown(f"Vertex AI backend (Project: {PROJECT_ID}, Loc: {LOCATION})..."))
        if not PROJECT_ID or not LOCATION or PROJECT_ID == "your-gcp-project-id":
             raise ValueError("PROJECT_ID/LOCATION invalid for Vertex AI.")
        # Initialize Client for Vertex
        ai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
        display(Markdown(f"✅ Vertex AI Client Initialized."))
    else: # Gemini API Mode
        display(Markdown("Gemini API backend (using API Key)..."))
        effective_api_key = GEMINI_API_KEY if GEMINI_API_KEY != "YOUR_API_KEY_HERE" else os.environ.get("GOOGLE_API_KEY")
        if not effective_api_key:
             if os.environ.get("GOOGLE_API_KEY"): effective_api_key = None # Client uses env var
             else: raise ValueError("Gemini API Key required but not found.")
        # Initialize Client for Gemini API
        ai_client = genai.Client(api_key=effective_api_key, vertexai=False)
        display(Markdown(f"✅ Gemini API Client Initialized."))

except ValueError as ve: display(Markdown(f"❌ **Config Error:** {ve}")); ai_client = None
except Exception as e: display(Markdown(f"❌ **AI Client Error:** {e}.")); logger.error("AI Client Init Failed", exc_info=True); ai_client = None

# --- Initialize Storage Client (ONLY for Vertex AI mode) --- #
if USE_VERTEX:
    display(Markdown("### Initializing GCS Client (Vertex Mode Only)"))
    if not GCS_AVAILABLE: display(Markdown("❌ GCS lib missing.")); raise RuntimeError("Missing GCS lib.")
    if not GCS_BUCKET or GCS_BUCKET == "your-gcs-bucket-name": display(Markdown("❌ GCS_BUCKET needed.")); raise ValueError("GCS_BUCKET required.")
    try:
        storage_client = storage.Client(project=PROJECT_ID)
        if not storage_client.bucket(GCS_BUCKET).exists(): display(Markdown(f"⚠️ GCS Bucket `{GCS_BUCKET}` inaccessible."))
        else: display(Markdown(f"✅ GCS Client Initialized (Bucket: '{GCS_BUCKET}')."))
    except Exception as e:
        display(Markdown(f"❌ **GCS Client Error:** {e}.")); logger.error("GCS Client Init Failed", exc_info=True)
        if not SKIP_PREPARE: raise RuntimeError("GCS client failed.")
        else: display(Markdown("⚠️ GCS client failed, but skipping prep."))
else:
    display(Markdown("### Initializing Gemini API Client (File API)"))
    try:
        storage_client = ai_client.files
    except Exception as e:
        display(Markdown(f"❌ **Gemini File API Client Error:** {e}.")); logger.error("Gemini API Client Init Failed", exc_info=True)
    display(Markdown(f"✅ Gemini File API Client Initialized."))

# --- Final Checks --- #
if ai_client is None: raise RuntimeError("AI client failed.")
if USE_VERTEX and storage_client is None and not SKIP_PREPARE: raise RuntimeError("GCS client failed for Vertex prep.")
display(Markdown("✅ Client initialization complete."))


## Utility Functions

In [ ]:
# --- File/Data Handling ---
def load_processed_qids(filename: str) -> Set[str]:
    processed_qids = set()
    if Path(filename).is_file():
        try:
            df = pd.read_csv(filename, usecols=['qid'], dtype={'qid': str}, on_bad_lines='warn')
            processed_qids = set(df['qid'].dropna().unique())
            logger.info(f"Loaded {len(processed_qids)} processed QIDs from {filename}")
        except Exception as e:
            logger.warning(f"Could not read QIDs from {filename}: {e}. Assuming zero processed.")
    return processed_qids

def download_file_with_progress(url: str, destination: Path):
    logger.info(f"Downloading {url} to {destination}...")
    try:
        response = requests.get(url, stream=True, timeout=600)
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024 * 1024
        with open(destination, 'wb') as f, tqdm(
            desc=f"Downloading {destination.name}", total=total_size, unit='iB', unit_scale=True, unit_divisor=1024
        ) as bar:
            for data in response.iter_content(block_size):
                size = f.write(data)
                bar.update(size)
        if total_size != 0 and bar.n != total_size:
            destination.unlink(missing_ok=True)
            raise RuntimeError(f"Download size mismatch for {destination.name}.")
        logger.info(f"Successfully downloaded {destination}")
    except Exception as e:
        destination.unlink(missing_ok=True)
        logger.error(f"Download failed for {url}: {e}")
        raise

def extract_zip(zip_path: Path, extract_to: Path):
    logger.info(f"Extracting {zip_path.name} to {extract_to}...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            members = [m for m in zip_ref.namelist() if not m.startswith('__MACOSX/') and not m.endswith('.DS_Store')]
            with tqdm(total=len(members), desc=f"Extracting {zip_path.name}") as pbar:
                for member in members:
                    zip_ref.extract(member=member, path=extract_to)
                    pbar.update(1)
        logger.info(f"Successfully extracted {zip_path} to {extract_to}")
    except Exception as e:
        logger.error(f"Extraction error: {e}")
        raise
    
def move_videos_to_main_directory(base_path):
    """Find all MP4 files in subdirectories and move them to the main directory."""
    logger.info(f"Moving all videos to main directory: {base_path}")
    moved_count = 0
    failed_count = 0
    
    # Find all MP4 files in subdirectories (but not in the main directory)
    for file_path in list(base_path.glob('**/*.mp4')):
        # Skip files already in the main directory or hidden Mac files
        if file_path.parent == base_path or file_path.name.startswith('._'):
            continue
            
        # Destination in the main directory
        dest_path = base_path / file_path.name
        
        try:
            # Move the file
            shutil.move(str(file_path), str(dest_path))
            moved_count += 1
            if moved_count % 50 == 0:
                logger.info(f"Moved {moved_count} videos so far...")
        except Exception as e:
            logger.error(f"Error moving {file_path}: {e}")
            failed_count += 1
    
    logger.info(f"Moved {moved_count} videos to main directory. Failed: {failed_count}")
    

def create_or_update_metadata(metadata_path: str, dataset_df: pd.DataFrame, video_updates: Dict[str, Dict]):
    try:
        required_cols = ['video_id', 'qid']
        update_cols = ['local_path', 'gcs_uri', 'file_api_name', 'status']
        dtype_map = {'video_id': str, 'qid': str} # Ensure IDs are strings

        if not Path(metadata_path).is_file():
            logger.info(f"Creating metadata file: {metadata_path}")
            meta_df = dataset_df.copy()
            for col in update_cols: meta_df[col] = pd.NA
            meta_df['status'] = 'pending'
        else:
            logger.debug(f"Loading existing metadata: {metadata_path}")
            meta_df = pd.read_csv(metadata_path, dtype=dtype_map)
            for col in update_cols: # Add missing update columns if needed
                 if col not in meta_df.columns: meta_df[col] = pd.NA

        if not all(col in meta_df.columns for col in required_cols):
            raise ValueError(f"Metadata missing required columns ({required_cols}).")

        updates_df = pd.DataFrame.from_dict(video_updates, orient='index')
        updates_df.index.name = 'video_id'
        updates_df.reset_index(inplace=True)
        updates_df['video_id'] = updates_df['video_id'].astype(str)

        # Use merge for robust updating across potentially multiple rows per video_id
        # First, prepare updates DF with only the necessary columns (video_id + update_cols)
        merge_cols = ['video_id'] + [col for col in update_cols if col in updates_df.columns]
        updates_to_merge = updates_df[merge_cols].drop_duplicates(subset=['video_id'], keep='last')

        # Merge, prioritizing updates
        # Suffixes help identify original vs update cols if needed, but update will overwrite
        merged_df = pd.merge(meta_df, updates_to_merge, on='video_id', how='left', suffixes=('', '_update'))

        # Apply the updates
        for col in update_cols:
            update_col_name = col + '_update'
            if update_col_name in merged_df.columns:
                # Fill NAs in original col with update col, then drop update col
                meta_df[col] = merged_df[update_col_name].fillna(merged_df[col])
                # Alternative: Directly update where update is not NA
                # meta_df[col] = np.where(merged_df[update_col_name].notna(), merged_df[update_col_name], merged_df[col])

        meta_df.to_csv(metadata_path, index=False, encoding='utf-8')
        logger.info(f"Metadata file '{metadata_path}' updated with {len(video_updates)} video records.")

    except Exception as e:
        logger.error(f"Error updating metadata {metadata_path}: {e}", exc_info=True)
        raise

def load_metadata_for_inference(metadata_file: str = METADATA_FILE) -> Dict[str, List[Dict]]:
    if not Path(metadata_file).is_file(): return {}
    video_questions = defaultdict(list)
    required_col = 'gcs_uri' if USE_VERTEX else 'file_api_name'
    try:
        df = pd.read_csv(metadata_file, dtype=str).fillna('')
        if 'video_id' not in df.columns or required_col not in df.columns:
            logger.error(f"Metadata missing 'video_id' or '{required_col}'.")
            return {}
        valid_df = df[df['video_id'].astype(bool) & df[required_col].astype(bool)]
        if len(valid_df) == 0:
             logger.warning(f"No videos found with '{required_col}' in {metadata_file}. Check Step 4.")
             return {}
        for video_id, group in valid_df.groupby('video_id'):
             video_questions[video_id] = group.to_dict('records')
        logger.info(f"Loaded {len(video_questions)} videos ({len(valid_df)} questions) with valid IDs for inference.")
        return dict(video_questions)
    except Exception as e:
        logger.error(f"Error loading metadata for inference: {e}", exc_info=True)
        return {}

# --- Upload/Verification Helpers ---
def upload_to_gcs(storage_client, bucket_name: str, source_file_path: Path, destination_blob_name: str) -> Optional[str]:
    if not GCS_AVAILABLE or storage_client is None or not source_file_path.is_file(): return None
    try:
        blob = storage_client.bucket(bucket_name).blob(destination_blob_name)
        blob.upload_from_filename(str(source_file_path))
        gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"
        logger.debug(f"GCS OK: {source_file_path} -> {gcs_uri}")
        return gcs_uri
    except Exception as e:
        logger.error(f"GCS Fail: {source_file_path}. Error: {e}")
        return None

def upload_via_file_api(storage_client, local_path: Path, display_name: str) -> Optional[str]:
    if storage_client is None or not local_path.is_file(): return None
    try:
        logger.debug(f"Uploading {local_path} via File API...")
        uploaded_file = storage_client.upload(file=local_path)
        logger.info(f"File API OK: {local_path} -> {uploaded_file.name}")
        return uploaded_file.name
    except Exception as e:
        logger.error(f"File API Fail: {local_path}. Error: {e}", exc_info=True)
        return None

def verify_gcs_file_exists(storage_client, gcs_uri: str) -> bool:
    if not GCS_AVAILABLE or storage_client is None or not gcs_uri: return False
    try:
        exists = storage.Blob.from_string(gcs_uri, client=storage_client).exists()
        if not exists: logger.warning(f"GCS verify failed: {gcs_uri}")
        return exists
    except Exception as e:
        logger.error(f"Error verifying GCS {gcs_uri}: {e}")
        return False

def verify_file_api_resource_exists(storage_client, file_api_name: str) -> bool:
    if not storage_client or not file_api_name: return False
    try:
        _ = storage_client.get(name=file_api_name) # Sync get for verification
        return True
    except Exception as e:
        logger.error(f"Error verifying File API {file_api_name}: {e}")
        return False

def verify_local_file_exists(local_path: str) -> bool:
    exists = Path(local_path).is_file() if local_path else False
    if not exists: logger.warning(f"Local verify failed: {local_path}")
    return exists

# --- Prompt Building ---
def build_prompt(question_info: dict) -> str:
    question = question_info.get("question", "")
    q_type = question_info.get("question_type", "default")
    template = PROMPT_TEMPLATES.get(q_type, PROMPT_TEMPLATES["default"])
    # if q_type is MCQ
    if q_type == "Multiple-choice Question with a Single Correct Answer":
        return template.format(question=question).strip() + "\n" + "E. None of the above"
    return template.format(question=question).strip() + "\n" + question_info.get("question_prompt").strip()

# --- Rate Limiter ---
class AsyncRateLimiter:
    """
    An asyncio-compatible token bucket rate limiter.

    Args:
        rate (int): The maximum number of requests allowed per period.
        period (float): The time period in seconds (default: 60 for RPM).
        capacity (int, optional): The maximum burst capacity. Defaults to `rate`.
    """
    def __init__(self, rate: int, period: float = 60.0, capacity: Optional[int] = None):
        if rate <= 0:
            raise ValueError("Rate must be positive")
        if period <= 0:
            raise ValueError("Period must be positive")

        self.rate = rate
        self.period = float(period)
        self.capacity = float(capacity if capacity is not None else rate)
        self._tokens = self.capacity # Start full
        self._last_refill_time = time.monotonic()
        self._lock = asyncio.Lock()

    def _get_tokens_per_second(self) -> float:
        return self.rate / self.period

    async def _refill(self):
        """Replenishes tokens based on elapsed time. Must be called under lock."""
        now = time.monotonic()
        elapsed = now - self._last_refill_time
        if elapsed > 0:
            tokens_to_add = elapsed * self._get_tokens_per_second()
            self._tokens = min(self.capacity, self._tokens + tokens_to_add)
            self._last_refill_time = now

    async def acquire(self):
        """
        Acquires a token, waiting if necessary.
        """
        async with self._lock:
            await self._refill() # Refill based on time since last acquire/refill

            while self._tokens < 1:
                # Calculate how long to wait for 1 token
                tokens_needed = 1.0 - self._tokens
                wait_time = tokens_needed / self._get_tokens_per_second()

                # Release the lock before sleeping
                lock_released = True
                try:
                    self._lock.release()
                    logger.debug(f"Rate limit hit. Waiting for {wait_time:.3f}s for next token.")
                    await asyncio.sleep(wait_time)
                finally:
                    # Re-acquire the lock if it was released
                    if lock_released:
                        await self._lock.acquire()

                # Refill again after waiting, as more time has passed
                await self._refill()

            # Consume a token
            self._tokens -= 1.0


# Download, Extract & Prepare Dataset

## Fetch Dataset from HuggingFace

In [ ]:
dataset_path = Path(DATASET_CSV)

if dataset_path.is_file() and SKIP_FETCH:
    logger.info(f"Dataset file '{DATASET_CSV}' exists and SKIP_FETCH is True. Skipping.")
    display(Markdown(f"✅ Skipping fetch: Found existing `{DATASET_CSV}`."))
    # Load the existing dataframe for use in Step 2
    try:
        dataset_df = pd.read_csv(dataset_path, dtype=str) # Load all as string initially
        logger.info(f"Loaded existing dataset from {DATASET_CSV} ({len(dataset_df)} rows).")
    except Exception as e:
        logger.error(f"Failed to load existing dataset file {DATASET_CSV}: {e}")
        display(Markdown(f"❌ Error loading existing `{DATASET_CSV}`: {e}. Please delete the file or set SKIP_FETCH=False."))
        raise
else:
    logger.info(f"Fetching dataset '{HF_DATASET_NAME}' (split: '{HF_DATASET_SPLIT}') from HuggingFace...")
    try:
        dataset = load_dataset(HF_DATASET_NAME, split=HF_DATASET_SPLIT, cache_dir=HF_CACHE_DIR)
        dataset_df = dataset.to_pandas()
        # Ensure key columns are strings
        for col in ['qid', 'video_id', 'question', 'question_type']:
             if col in dataset_df.columns:
                 dataset_df[col] = dataset_df[col].astype(str)
        dataset_df.to_csv(dataset_path, index=False, encoding='utf-8')
        logger.info(f"Successfully fetched dataset and saved to {DATASET_CSV} ({len(dataset_df)} rows).")
        display(Markdown(f"✅ Dataset fetched and saved to `{DATASET_CSV}` ({len(dataset_df)} rows)."))
        display(dataset_df.head())
    except Exception as e:
        logger.error(f"Failed to fetch or save dataset: {e}", exc_info=True)
        display(Markdown(f"❌ **Error fetching dataset:** {e}. Check connection, dataset name/split, cache dir permissions."))
        raise RuntimeError("Dataset fetching failed. Cannot continue.")

# Ensure dataset_df is loaded if skipping fetch didn't load it (e.g., first run with skip=True and no file)
if 'dataset_df' not in locals():
    if dataset_path.is_file():
        try:
            dataset_df = pd.read_csv(dataset_path, dtype=str)
        except Exception as e:
            logger.error(f"Critical error: Could not load dataset from {DATASET_CSV} after attempting fetch/skip: {e}")
            raise
    else:
        raise RuntimeError(f"Critical error: Dataset DataFrame not loaded and file {DATASET_CSV} not found.")

## Download, Extract, and Prepare Videos

Downloads, extracts, and uploads videos (to GCS or File API). Updates `video_metadata.csv`.

In [ ]:
dataset_path = Path(DATASET_CSV)
dataset_df = pd.read_csv(dataset_path, dtype=str)

### Download Video Archive

In [ ]:
display(Markdown("### Downloading Archive"))
if zip_file_path.is_file() and SKIP_DOWNLOAD_ZIP:
    display(Markdown(f"✅ Skipping download: Found `{zip_file_path}`."))
else:
    try: download_file_with_progress(VIDEO_ZIP_URL, zip_file_path); display(Markdown(f"✅ Downloaded: `{zip_file_path}`."))
    except Exception as e:
        display(Markdown(f"❌ **Download Error:** {e}."))
        if not SKIP_EXTRACT or not SKIP_PREPARE: raise RuntimeError(f"Download failed.")
        else: display(Markdown("⚠️ Download failed, skipping steps."))

### Extract Video Archive

In [ ]:
display(Markdown("### Extracting Archive"))
if any(extracted_videos_path.glob('*.mp4')) and SKIP_EXTRACT:
    display(Markdown(f"✅ Skipping extraction: Files in `{extracted_videos_path}`."))
elif not zip_file_path.is_file():
    display(Markdown(f"❌ Cannot extract: `{zip_file_path}` missing."))
    if not SKIP_PREPARE: raise RuntimeError(f"Zip missing.")
    else: display(Markdown("⚠️ Extraction skipped (no zip)."))
else:
    try: 
        extract_zip(zip_file_path, extracted_videos_path)
        # Move all videos to main directory
        move_videos_to_main_directory(extracted_videos_path)
        display(Markdown(f"✅ Extracted to `{extracted_videos_path}` and moved all videos to main directory."))
    except Exception as e:
        display(Markdown(f"❌ **Extraction Error:** {e}."))
        if not SKIP_PREPARE: raise RuntimeError("Extraction failed.")
        else: display(Markdown("⚠️ Extraction failed, skipping prep."))

### Slow/Speed Up Videos
Losslessly change video speed while also re-encoding audio to maintain pitch. As
a result, is super fast. Could be made faster if using asyncio to concurrently run
ffmpeg.

In [ ]:
async def run_subprocess(cmd, check=True, capture_output=False):
    """Helper function to run subprocess asynchronously."""
    stdout_pipe = asyncio.subprocess.PIPE if capture_output else asyncio.subprocess.DEVNULL
    # Capture stderr only if check is True or capture_output is True, otherwise DEVNULL
    stderr_pipe = asyncio.subprocess.PIPE if check or capture_output else asyncio.subprocess.DEVNULL

    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=stdout_pipe,
        stderr=stderr_pipe
    )
    stdout, stderr = await process.communicate()

    if check and process.returncode != 0:
        error_msg = f"Command '{' '.join(cmd)}' failed with return code {process.returncode}"
        stderr_decoded = stderr.decode(errors='ignore') if stderr else ""
        if stderr_decoded:
            error_msg += f"\nStderr: {stderr_decoded}"
        # Raise specific exception to potentially capture stderr later
        raise subprocess.CalledProcessError(process.returncode, cmd, output=stdout, stderr=stderr)

    return stdout, stderr, process.returncode

async def process_single_video(vid_path, speed_videos_path, VIDEO_SPEED_FACTOR, semaphore):
    """Asynchronously processes a single video. Returns status string."""
    vid_path_str = str(vid_path.resolve())
    out_path = speed_videos_path / vid_path.name
    out_path_str = str(out_path.resolve())

    async with semaphore: # Limit concurrency
        if out_path.is_file():
            return 'skipped'

        if VIDEO_SPEED_FACTOR == 1.0:
            try:
                # Use asyncio.to_thread for potentially blocking I/O
                await asyncio.to_thread(shutil.copy, vid_path_str, out_path_str)
                return 'processed'
            except Exception as e:
                try:
                    logger.error(f"Error copying {vid_path.name}: {e}")
                except NameError:
                    print(f"Error copying {vid_path.name}: {e}")
                return 'error'

        # --- Process video with speed change ---
        tf_bitstream_path = None
        tf_audio_path = None
        tf_final_path = None
        try:
            # Create temporary files (synchronous part is okay here)
            # Context manager ensures files are closed before ffmpeg uses them
            with tempfile.NamedTemporaryFile(delete=False, suffix=".h264") as tf_b, \
                 tempfile.NamedTemporaryFile(delete=False, suffix=".aac") as tf_a, \
                 tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tf_f:
                tf_bitstream_name = tf_b.name
                tf_audio_name = tf_a.name
                tf_final_name = tf_f.name
            # Store paths for cleanup
            tf_bitstream_path = Path(tf_bitstream_name)
            tf_audio_path = Path(tf_audio_name)
            tf_final_path = Path(tf_final_name)


            # Get original FPS
            ffprobe_cmd = [
                "ffprobe", "-v", "error", "-select_streams", "v", "-of", "default=noprint_wrappers=1:nokey=1",
                "-show_entries", "stream=r_frame_rate", vid_path_str
            ]
            stdout, _, _ = await run_subprocess(ffprobe_cmd, check=True, capture_output=True)
            fps = float(fractions.Fraction(stdout.decode().strip()))
            new_fps = fps * VIDEO_SPEED_FACTOR

            # Extract and speed up audio
            factor = VIDEO_SPEED_FACTOR
            filter_parts = []
            while factor > 2.0:
                filter_parts.append("atempo=2.0")
                factor /= 2.0
            while factor < 0.5:
                filter_parts.append("atempo=0.5")
                factor /= 0.5
            if abs(factor - 1.0) > 1e-6:
                 filter_parts.append(f"atempo={factor:.6f}")

            if not filter_parts:
                 audio_cmd = ["ffmpeg", "-y", "-i", vid_path_str, "-vn", "-c:a", "copy", tf_audio_name]
            else:
                audio_filter = ",".join(filter_parts)
                audio_cmd = ["ffmpeg", "-y", "-i", vid_path_str, "-vn", "-filter:a", audio_filter, "-c:a", "aac", "-b:a", "128k", tf_audio_name]
            await run_subprocess(audio_cmd, check=True)


            # Extract h264 bitstream
            extract_cmd = ["ffmpeg", "-y", "-i", vid_path_str, "-map", "0:v", "-c:v", "copy", "-bsf:v", "h264_mp4toannexb", tf_bitstream_name]
            await run_subprocess(extract_cmd, check=True)

            # Remux bitstream with new audio and FPS
            remux_cmd = ["ffmpeg", "-y", "-fflags", "+genpts", "-r", f"{new_fps:.6f}", "-i", tf_bitstream_name, "-i", tf_audio_name, "-map", "0:v", "-map", "1:a", "-c:v", "copy", "-c:a", "copy", tf_final_name]
            await run_subprocess(remux_cmd, check=True)

            # Move final file (use asyncio.to_thread)
            await asyncio.to_thread(shutil.move, tf_final_name, out_path_str)
            return 'processed'

        except Exception as e:
            err_msg = f"Error processing {vid_path.name}: {e}"
            # Include ffmpeg stderr if available
            if isinstance(e, subprocess.CalledProcessError) and e.stderr:
                 err_msg += f"\nFFmpeg/FFprobe Stderr:\n{e.stderr.decode(errors='ignore')}"
            try:
                logger.error(err_msg)
            except NameError:
                print(err_msg)
            return 'error'
        finally:
            # Clean up temporary files asynchronously using to_thread
            async def _cleanup():
                if tf_bitstream_path and tf_bitstream_path.exists():
                    tf_bitstream_path.unlink(missing_ok=True)
                if tf_audio_path and tf_audio_path.exists():
                    tf_audio_path.unlink(missing_ok=True)
                # tf_final is moved, only delete if error occurred before move
                if tf_final_path and tf_final_path.exists():
                    tf_final_path.unlink(missing_ok=True)
            # Run sync cleanup in thread only if paths were assigned
            if tf_bitstream_path or tf_audio_path or tf_final_path:
                 await asyncio.to_thread(_cleanup)


# --- Main Cell Logic ---

async def run_processing(): # Wrap in an async function to use await
    display(Markdown("### Preparing Videos"))
    if dataset_df is None:
        raise RuntimeError("Dataset DF unavailable.")

    # Ensure output directory exists
    speed_videos_path.mkdir(parents=True, exist_ok=True)

    all_video_ids = sorted(list(dataset_df['video_id'].dropna().unique()))
    # Use logging if available, otherwise print
    try:
        logger.info(f"Processing {len(all_video_ids)} unique video IDs.")
    except NameError:
        print(f"Processing {len(all_video_ids)} unique video IDs.")

    vid_paths = list(extracted_videos_path.glob("*.mp4"))

    # Limit concurrency
    concurrency_limit = MAX_ASYNC_WORKERS
    try:
        logger.info(f"Using concurrency limit: {concurrency_limit}")
    except NameError:
        print(f"Using concurrency limit: {concurrency_limit}")
    semaphore = asyncio.Semaphore(concurrency_limit)

    tasks = []
    # Keep the familiar loop structure for creating tasks
    print(f"Preparing tasks for {len(vid_paths)} videos...")
    for vid_path in vid_paths:
         # Create a task for each video processing job
         # Pass necessary arguments to the task creator
         task = asyncio.create_task(process_single_video(vid_path, speed_videos_path, VIDEO_SPEED_FACTOR, semaphore))
         tasks.append(task)

    # Now, run all the created tasks concurrently and display progress
    # Use asyncio.as_completed with a standard tqdm progress bar
    print(f"Transforming {len(tasks)} Videos...")
    results = []
    # Use the imported tqdm (now tqdm.auto) to create a standard progress bar instance
    with tqdm(total=len(tasks), desc="Transforming Videos", unit="video") as pbar:
        for future in asyncio.as_completed(tasks):
            try:
                result = await future # Get result from completed task
                results.append(result)
            except Exception as exc:
                # Log errors from tasks that failed internally if not caught by process_single_video
                # (process_single_video should ideally return 'error' status instead of raising)
                try:
                    logger.error(f"Task for a video failed: {exc}")
                except NameError:
                    print(f"Task for a video failed: {exc}")
                results.append('error') # Count as error if task itself fails unexpectedly
            finally:
                 pbar.update(1) # Increment progress bar regardless of outcome


    # Count results
    processed = results.count('processed')
    skipped = results.count('skipped')
    errors = results.count('error')

    print(f"\n\n{skipped} videos skipped, {processed} videos processed, {errors} errors, {len(vid_paths)} total.")

# --- Execute the async processing ---
# In a Jupyter Notebook, you usually need to await the top-level async function.
# If top-level await isn't enabled, you might need nest_asyncio or run manually.
# Using await directly is the most common way in modern notebooks.
await run_processing()


### Preparing and Upload Videos to GCS or File API

In [ ]:
# --- Prepare Videos (Upload GCS/File API) & Update Metadata --- #
#-- Can set SKIP_PREPARE to True to skip this step if you already uploaded it into bucket through Vertex AI Mode/ into files through Gemini API key --#
display(Markdown("### Preparing Videos & Updating Metadata"))
if SKIP_PREPARE:
    display(Markdown("✅ Skipping video preparation."))
elif storage_client is None:
     display(Markdown("❌ Cannot prepare: Client not ready.")); raise RuntimeError("Client missing.")
else:
    if dataset_df is None: raise RuntimeError("Dataset DF unavailable.")
    all_video_ids = sorted(list(dataset_df['video_id'].dropna().unique()))
    logger.info(f"Processing {len(all_video_ids)} unique video IDs.")

    videos_to_process_ids = all_video_ids
    if MAX_VIDEOS_TO_PROCESS is not None:
        videos_to_process_ids = all_video_ids[:MAX_VIDEOS_TO_PROCESS]
        logger.info(f"Limiting to {len(videos_to_process_ids)} videos.")

    # Load existing metadata to check status
    existing_statuses = {}
    resource_ids = {}
    required_id_col = 'gcs_uri' if USE_VERTEX else 'file_api_name'
    if Path(METADATA_FILE).is_file():
        try:
            existing_df = pd.read_csv(METADATA_FILE, dtype=str)
            if 'video_id' in existing_df.columns and 'status' in existing_df.columns:
                existing_statuses = pd.Series(existing_df.status.values, index=existing_df.video_id).to_dict()
            if 'video_id' in existing_df.columns and required_id_col in existing_df.columns:
                 resource_ids = pd.Series(existing_df[required_id_col].values, index=existing_df.video_id).dropna().to_dict()
            logger.info("Checked existing metadata statuses/IDs.")
        except Exception as e: logger.warning(f"Could not load existing metadata: {e}")

    video_metadata_updates = {}
    processed_count, upload_failures, missing_local, skipped_count = 0, 0, 0, 0
    num_batches = math.ceil(len(videos_to_process_ids) / UPLOAD_BATCH_SIZE_GCS)
    prep_mode = "GCS Upload" if USE_VERTEX else "File API Upload"

    with tqdm(total=len(videos_to_process_ids), desc=f"Preparing ({prep_mode})") as pbar:
        for i in range(0, len(videos_to_process_ids), UPLOAD_BATCH_SIZE_GCS):
            batch_ids = videos_to_process_ids[i : i + UPLOAD_BATCH_SIZE_GCS]
            batch_num = (i // UPLOAD_BATCH_SIZE_GCS) + 1
            logger.info(f"Prep Batch {batch_num}/{num_batches}...")
            current_batch_updates = {}

            for video_id in batch_ids:
                pbar.set_postfix_str(f"ID: {video_id}")
                update_data = {"local_path": None, "gcs_uri": None, "file_api_name": None, "status": "error_unknown"}
                local_video_path = speed_videos_path / f"{video_id}.mp4"
                current_status = existing_statuses.get(video_id, 'pending')
                existing_resource_id = resource_ids.get(video_id)
                is_already_processed = False

                # Check if already uploaded and verified
                if current_status in ['uploaded_gcs', 'uploaded_file_api'] and existing_resource_id:
                     verified = False
                     if USE_VERTEX: verified = verify_gcs_file_exists(storage_client, existing_resource_id)
                     else: verified = verify_file_api_resource_exists(storage_client, existing_resource_id)
                     if verified:
                         logger.debug(f"Skipping verified video {video_id} ('{current_status}').")
                         is_already_processed = True
                         skipped_count += 1
                         update_data.update({ # Ensure metadata is consistent
                             'local_path': str(local_video_path) if local_video_path.is_file() else None,
                             'status': current_status,
                             required_id_col: existing_resource_id
                         })
                     else:
                         logger.warning(f"Video {video_id} ({current_status}) needs re-processing (verification failed).")
                elif current_status != 'pending':
                     logger.debug(f"Video {video_id} has non-pending status '{current_status}' but no verified resource ID. Re-processing.")

                if is_already_processed:
                    processed_count += 1
                    current_batch_updates[video_id] = update_data
                    pbar.update(1)
                    continue

                # Process if needed
                if local_video_path.is_file():
                    update_data["local_path"] = str(local_video_path)
                    resource_id_result = None
                    if USE_VERTEX:
                        blob_name = f"videos/{video_id}.mp4"
                        resource_id_result = upload_to_gcs(storage_client, GCS_BUCKET, local_video_path, blob_name)
                        if resource_id_result: update_data.update({"gcs_uri": resource_id_result, "status": "uploaded_gcs"})
                        else: update_data["status"] = "gcs_upload_failed"; upload_failures += 1
                    else: # Gemini API
                        resource_id_result = upload_via_file_api(storage_client, local_video_path, f"vid_{video_id}")
                        if resource_id_result: update_data.update({"file_api_name": resource_id_result, "status": "uploaded_file_api"})
                        else: update_data["status"] = "file_api_upload_failed"; upload_failures += 1
                else:
                    logger.warning(f"Local file missing: {local_video_path}")
                    missing_local += 1
                    update_data["status"] = "local_missing"

                current_batch_updates[video_id] = update_data
                processed_count += 1
                pbar.update(1)

            # Update metadata after batch
            if current_batch_updates:
                 try: create_or_update_metadata(METADATA_FILE, dataset_df, current_batch_updates)
                 except Exception as e: logger.error(f"Metadata update failed batch {batch_num}: {e}")
                 video_metadata_updates.update(current_batch_updates)

    logger.info(f"Prep finished. Checked: {processed_count}, Skipped(verified): {skipped_count}, Missing Local: {missing_local}, Upload Failures: {upload_failures}")
    display(Markdown(f"✅ Video preparation complete. See logs. Metadata: `{METADATA_FILE}`."))


# Bulk Inference 
## (Agentic) (Turn by turn) (Generated Questions) Bulk inference CoCoT reasoning with summary model

In [ ]:
RESULTS_FINAL_DIR = os.path.join(f"all_results/full_inference_CoCoT_generated_questions/{MODEL_NAME}", RESULTS_FILE)
os.makedirs(os.path.dirname(RESULTS_FINAL_DIR), exist_ok=True)

In [ ]:
ANSWERS_DIR = f"generated_questions/{QUESTIONS_MODEL_NAME}/chat_history"

In [ ]:
def deserialize_chat(chat_json):
    return [
        types.Content(
            role=msg["role"],
            parts=[types.Part.from_text(text=msg["parts"][0])]
        ) for msg in chat_json
    ]

In [ ]:
# --- Inference Functions ---

async def perform_inference_single_async(
    question_info: Dict,
    client: Any,
    semaphore: asyncio.Semaphore,
    rate_limiter: Optional[AsyncRateLimiter],
    results_queue: asyncio.Queue
) -> None: # Return None as result is put in queue
    """
    Async inference for one question, putting the result into a queue.
    """
    qid = question_info.get("qid", "?")
    video_id = question_info.get("video_id", "?")
    prompt_text = build_prompt(question_info)
    gcs_uri = question_info.get("gcs_uri")
    file_api_name = question_info.get("file_api_name")
    start_time = time.time()
    result = None # Default result in case of early exit
    
    chat_path = os.path.join(ANSWERS_DIR,video_id+".json")
    if not os.path.isfile(chat_path):
        logger.error(f"QID {qid} (Async): Chat file not found for video ID {video_id}.")
        result = {"qid": qid, "pred": f"ERROR: Chat file not found for video ID {video_id}.", "duration": 0, "finish_reason": "N/A", "status": "Failed (Chat File Not Found)"}
        await results_queue.put(result)
        return
    
    
    # Load chat history with generated questions from JSON file
    with open(chat_path) as f:
        chat_json = json.load(f)
        chat = deserialize_chat(chat_json)

    # --- Prepare Inputs (Same as before) ---
    try:
        video_part = None
        if USE_VERTEX:
            # ... (GCS URI logic) ...
            if not gcs_uri: raise ValueError("Missing GCS URI.")
            video_part = types.Part.from_uri(mime_type='video/mp4', file_uri=gcs_uri)
        else: # Gemini API Mode
            # ... (File API get logic) ...
             if not file_api_name: raise ValueError("Missing File API name.")
             try:
                 file_object = await client.aio.files.get(name=file_api_name)
                 video_part = file_object
             except genai_errors.NotFoundError: raise FileNotFoundError(f"File API '{file_api_name}' not found.")
             except Exception as e: raise RuntimeError(f"Failed get File API obj: {e}")
        
        user_msg = types.Content(role="user",
                            parts=[types.Part.from_text(text=prompt_text)])

        # Merge video part, chat history, and user message
        if video_part is None: raise RuntimeError("Video part preparation failed.")
        contents = [video_part] + chat + [user_msg]

    except (ValueError, FileNotFoundError, RuntimeError) as e:
        logger.error(f"QID {qid} (Async): Input Error - {e}")
        # Put error result in queue
        result = {"qid": qid, "pred": f"ERROR: Input Fail - {e}", "duration": 0, "finish_reason": "N/A", "status": "Failed (Input)"}
        await results_queue.put(result)
        return # Exit the function
    except Exception as e:
         logger.error(f"QID {qid} (Async): Unexpected Input Prep Error: {e}", exc_info=True)
         result = {"qid": qid, "pred": f"ERROR: Input Prep Failed Unexpectedly - {e}", "duration": 0, "finish_reason": "N/A", "status": "Failed (Input Prep)"}
         await results_queue.put(result)
         return # Exit the function

    # --- Perform Inference with Retries, Semaphore, and Rate Limiting ---
    async with semaphore:
        for attempt in range(MAX_RETRIES + 1):
            try:
                if rate_limiter: await rate_limiter.acquire()
                api_start = time.time()
                logger.debug(f"QID {qid} (Async): Attempt {attempt + 1} sending request...")
                response = await client.aio.models.generate_content(
                    model=MODEL_NAME,
                    contents=contents,
                    config=CONFIG
                )

                # Process Response
                answer, reason, status, err_detail = "ERROR", "UNKNOWN", "Success", ""
                try:
                    answer = response.text.strip()
                    if response.candidates and hasattr(response.candidates[0], 'finish_reason') and response.candidates[0].finish_reason is not None:
                        reason = response.candidates[0].finish_reason.name
                except ValueError as ve:
                    status, err_detail = "Blocked/Empty", f"ValueError: {ve}. "
                    # ... (block/safety reason extraction) ...
                    answer = f"ERROR: {status}. {err_detail}"

                result = {"qid": qid, "pred": answer, "duration": time.time()-start_time, "finish_reason": reason, "status": status}
                await results_queue.put(result)
                logger.debug(f"QID {qid} (Async): Attempt {attempt + 1} {status} ({time.time()-api_start:.2f}s API / {time.time()-start_time:.2f}s Total). Result queued.")
                return # Exit function after success


            except Exception as e:
                 result = {"qid": qid, "pred": f"ERROR: - {e}", "duration": time.time()-start_time, "finish_reason": reason, "status": "Failed (Unexpected Error)"}
                 await results_queue.put(result)
                 return

        # Fallback (Should not be reached if logic above is correct)
        logger.error(f"QID {qid} (Async): Exited retry loop unexpectedly.")
        result = {"qid": qid, "pred": "ERROR: Unknown after retries", "duration": time.time()-start_time, "finish_reason": "UNKNOWN", "status": "Failed (Unknown)"}
        await results_queue.put(result)

async def results_writer_task(
    queue: asyncio.Queue,
    filename: str,
    write_batch_size: int = 20, # How many results to buffer before writing
    write_interval_sec: float = 10.0 # Max time between writes
):
    """Gets results from queue and writes them to CSV in batches."""
    results_buffer = []
    last_write_time = time.monotonic()
    # Define expected header based on the dict keys put in the queue
    fieldnames = ["qid", "pred", "status", "duration_sec", "finish_reason"]
    file_exists = Path(filename).is_file()

    logger.info(f"Writer task started. Writing results to {filename}")

    while True:
        try:
            # Wait for an item with a timeout
            result = await asyncio.wait_for(queue.get(), timeout=write_interval_sec)

            if result is None: # Signal to terminate
                logger.info("Writer task received termination signal.")
                break

            if isinstance(result, dict):
                 # Ensure duration is rounded here before adding to buffer
                 result["duration_sec"] = round(result.get("duration", -1), 2)
                 # Remove the original 'duration' key if desired
                 result.pop('duration', None)
                 results_buffer.append(result)
            else:
                logger.warning(f"Writer task received non-dict item: {result}")

            queue.task_done() # Signal that the item was processed

        except asyncio.TimeoutError:
            # Timeout occurred, write buffer if not empty, even if batch size not reached
            logger.debug("Writer task timeout reached.")
            pass # Continue to buffer check below

        except Exception as e:
            logger.error(f"Writer task encountered error getting from queue: {e}", exc_info=True)
            # Decide if this is fatal or if we should try to continue
            await asyncio.sleep(1) # Prevent fast spinning on error
            continue # Try to continue processing

        # Check if we should write the buffer
        buffer_size = len(results_buffer)
        time_since_last_write = time.monotonic() - last_write_time
        should_write = (
             buffer_size > 0 and
             (buffer_size >= write_batch_size or time_since_last_write >= write_interval_sec)
        )

        if should_write:
            logger.info(f"Writing batch of {buffer_size} results to {filename}...")
            try:
                # Use 'with open' for proper handling
                with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    # Write header only if file didn't exist at the start
                    if not file_exists:
                        writer.writeheader()
                        file_exists = True # Prevent writing header again
                    writer.writerows(results_buffer)

                results_buffer = [] # Clear buffer after successful write
                last_write_time = time.monotonic()
                logger.info(f"Batch written successfully.")
            except IOError as e:
                logger.error(f"IOError writing results batch to {filename}: {e}. Results may be lost.")
                # Optional: Add retry logic here, or store failed batch elsewhere
            except Exception as e:
                logger.error(f"Unexpected error writing results batch: {e}", exc_info=True)


    # --- Cleanup: Write any remaining items after receiving None signal ---
    if results_buffer:
        logger.info(f"Writing final remaining {len(results_buffer)} results...")
        try:
            with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                if not file_exists: writer.writeheader() # Check again in case file was deleted mid-run
                writer.writerows(results_buffer)
            logger.info("Final results written.")
        except Exception as e:
            logger.error(f"Error writing final results batch: {e}")

    logger.info("Writer task finished.")

# --- Modified Main Execution Function with UI ---
async def run_bulk_inference_async():
    """Runs the bulk inference process asynchronously with queue writer, rate limiting, and UI."""

    # --- Standard Setup ---
    if ai_client is None: logger.error("AI Client not initialized."); display(Markdown("❌ AI Client not initialized.")); return

    # --- Load Data and Prepare Tasks ---
    logger.info("Loading metadata for inference...")
    video_questions_for_inference = load_metadata_for_inference(METADATA_FILE)
    if not video_questions_for_inference: logger.warning("No video data ready."); return
    processed_qids = load_processed_qids(RESULTS_FINAL_DIR)

    inference_tasks_input = []
    tasks_skipped = 0
    required_col = 'gcs_uri' if USE_VERTEX else 'file_api_name'
    for video_id, questions in video_questions_for_inference.items():
        for question_info in questions:
             qid = question_info.get('qid')
             if not qid or qid in processed_qids or not question_info.get(required_col):
                 tasks_skipped += 1; continue
             inference_tasks_input.append(question_info)

    total_tasks = len(inference_tasks_input)
    logger.info(f"Prepared {total_tasks} new inference tasks. Skipped {tasks_skipped}." )
    if total_tasks == 0: logger.info("No new questions to process."); return

    # --- Setup Rate Limiter ---
    rate_limiter = None
    if REQUESTS_PER_MINUTE is not None and REQUESTS_PER_MINUTE > 0:
        capacity = 10 # Example: Keep burst capacity moderate
        rate_limiter = AsyncRateLimiter(rate=REQUESTS_PER_MINUTE, period=60.0, capacity=capacity)
        logger.info(f"Rate limiting enabled: {REQUESTS_PER_MINUTE} RPM, Capacity: {capacity}")
    else:
        logger.info("Rate limiting disabled.")

    # --- Create Queue and Start Writer Task ---
    results_queue = asyncio.Queue()
    writer_handle = asyncio.create_task(
        results_writer_task(results_queue, RESULTS_FINAL_DIR)
    )

    # --- Schedule and Run Inference Tasks with Progress Bar ---
    logger.info(f"Starting async inference for {total_tasks} questions (Concurrency: {MAX_ASYNC_WORKERS})...")
    semaphore = asyncio.Semaphore(MAX_ASYNC_WORKERS)
    start_bulk_time = time.time()

    # Create coroutines
    inference_coroutines = [
        perform_inference_single_async(q_info, ai_client, semaphore, rate_limiter, results_queue)
        for q_info in inference_tasks_input
    ]

    # --- Use asyncio.as_completed with tqdm_notebook for live progress ---
    completed_count = 0
    gather_exception = None
    try:
        # Create a future for each coroutine to track completion
        tasks = [asyncio.ensure_future(coro) for coro in inference_coroutines]
        # Use tqdm with as_completed
        for future in tqdm(asyncio.as_completed(tasks), total=total_tasks, desc="Async Inference"):
            try:
                await future # Wait for the next task to complete, raise exception if task failed
                completed_count += 1
            except Exception as task_exc:
                 # Log error from individual task if it wasn't caught inside
                 logger.error(f"Error surfaced from an inference task: {task_exc}", exc_info=False)
                 # Optionally decide if you want to stop processing other tasks
                 # gather_exception = task_exc # Store first exception
                 # break # Or continue processing others
    except Exception as outer_exc:
        # Catch errors during the setup/iteration of as_completed itself
        logger.error(f"Error during as_completed processing: {outer_exc}", exc_info=True)
        gather_exception = outer_exc

    # Ensure all tasks are awaited even if we broke early due to an error in one task
    # (This might be redundant if as_completed handles cancellation correctly, but safer)
    await asyncio.gather(*tasks, return_exceptions=True)

    bulk_duration = time.time() - start_bulk_time
    logger.info(f"Async inference task processing finished in {bulk_duration:.2f} seconds. Completed: {completed_count}/{total_tasks}.")
    if gather_exception:
         logger.error(f"Bulk inference encountered errors: {gather_exception}")

    # --- Signal Writer Task to Finish (ALWAYS DO THIS) ---
    logger.info("Signaling writer task to complete...")
    await results_queue.put(None)

    # --- Wait for Writer Task to Finish (ALWAYS DO THIS) ---
    logger.info("Waiting for writer task to finish writing remaining results...")
    try:
        await writer_handle # Wait until the writer processes the None signal and exits
        logger.info("Writer task has finished.")
    except Exception as writer_exc:
        logger.error(f"Error waiting for writer task: {writer_exc}", exc_info=True)

    # --- Final Summary ---
    logger.info(f"Bulk inference process complete. See logs and {RESULTS_FINAL_DIR}.")
    # Remove handler to prevent duplicate logs on re-run

# --- Run the async function ---
asyncio.run(run_bulk_inference_async())

Check with the clean up again after finishing the Bulk Inference

## Interim Up

In [ ]:
def interim_cleanup_and_report():
    """Final cleanup and report of results."""

    try:
        results_df = pd.read_csv(RESULTS_FINAL_DIR, dtype=str)
        logger.info(f"Loaded results from {RESULTS_FINAL_DIR} ({len(results_df)} rows).")
        
        # Correctly identify failed entries
        failed = results_df[results_df['CoT'].isna() | (results_df['status'] != 'Success')]
        
        # Success is everything not in failed
        success = results_df[~results_df.index.isin(failed.index)]

        display(Markdown(f"### Results Summary: {len(success)} Success, {len(failed)} Failed."))

        # Sort by 'qid'
        results_df.sort_values(by=['qid'], inplace=True)

        # remove duplicates based on 'qid' and keep the first occurrence
        results_df.drop_duplicates(subset=['qid'], keep='first', inplace=True)
        logger.info(f"Removed duplicates, remaining {len(results_df)} unique QIDs.")

        # Save the cleaned-up results to a new file with datetime suffix
        original_filename = Path(RESULTS_FINAL_DIR).stem
        # Get the directory of the original file
        original_dir = Path(RESULTS_FINAL_DIR).parent
        # Create the new file names
        success_filename = f"{original_filename}.csv"
        failed_filename = f"{original_filename}_failed.csv"
        # Create the full paths for the new files
        success_filename = original_dir / success_filename
        failed_filename = original_dir / failed_filename
        # Save the success and failed DataFrames to separate CSV files
        success.to_csv(success_filename, index=False, encoding='utf-8')
        failed.to_csv(failed_filename, index=False, encoding='utf-8')
     
    except Exception as e:
        logger.error(f"Error processing results file: {e}", exc_info=True)
        display(Markdown(f"❌ Error processing results file: {e}."))
        return

interim_cleanup_and_report()

# Cleanup and reporting

### Report With Summary Model

In [ ]:
SUMMARY_OUTPUT_CSV = RESULTS_FINAL_DIR.replace(".csv", "_cot_summary.csv")


def results_cleanup_and_report():
    """Final cleanup and report of results."""
    if not Path(RESULTS_FINAL_DIR).is_file(): display(Markdown(f"❌ Results file `{RESULTS_FINAL_DIR}` not found.")); return

    try:
        results_df = pd.read_csv(RESULTS_FINAL_DIR, dtype=str)
        logger.info(f"Loaded results from {RESULTS_FINAL_DIR} ({len(results_df)} rows).")
        
        success = results_df[results_df['status'] == 'Success']
        failed = results_df[results_df['status'] != 'Success']
        logger.info(f"Results Summary: {len(success)} Success, {len(failed)} Failed.")
        display(Markdown(f"### Results Summary: {len(success)} Success, {len(failed)} Failed."))

        avg_duration = results_df['duration_sec'].astype(float).mean()
        logger.info(f"Average duration for successful tasks: {avg_duration:.2f} seconds.")
        display(Markdown(f"### Average Duration: {avg_duration:.2f} seconds."))

        # Sort by 'qid'
        results_df.sort_values(by=['qid'], inplace=True)

        # remove duplicates based on 'qid' and keep the first occurrence
        results_df.drop_duplicates(subset=['qid'], keep='first', inplace=True)
        logger.info(f"Removed duplicates, remaining {len(results_df)} unique QIDs.")

        # remove all unnecessary columns
        columns_to_keep = ['qid', 'pred']
        results_df = results_df[columns_to_keep]

        new_name = SUMMARY_OUTPUT_CSV.replace('.csv', "_") +  time.strftime("%Y%m%d_%H%M%S") + ".csv"
        # Save the cleaned DataFrame to a CSV file
        results_df.to_csv(new_name, index=False, encoding='utf-8')
        logger.info(f"Cleaned results saved to {new_name}.")
        display(Markdown(f"✅ Cleaned results saved to `{new_name}`."))
     
    except Exception as e:
        logger.error(f"Error processing results file: {e}", exc_info=True)
        display(Markdown(f"❌ Error processing results file: {e}."))
        return
  
results_cleanup_and_report()

### Getting COT Thinking Process Only 

In [ ]:
COT_ONLY_OUTPUT_CSV = RESULTS_FINAL_DIR.replace(".csv", "_cot_only.csv")

def results_cleanup_and_report():
    """Final cleanup and report of results."""
    if not Path(RESULTS_FINAL_DIR).is_file(): display(Markdown(f"❌ Results file `{RESULTS_FINAL_DIR}` not found.")); return

    try:
        results_df = pd.read_csv(RESULTS_FINAL_DIR, dtype=str)
        logger.info(f"Loaded results from {RESULTS_FINAL_DIR} ({len(results_df)} rows).")
        
        success = results_df[results_df['status'] == 'Success']
        failed = results_df[results_df['status'] != 'Success']
        logger.info(f"Results Summary: {len(success)} Success, {len(failed)} Failed.")
        display(Markdown(f"### Results Summary: {len(success)} Success, {len(failed)} Failed."))

        avg_duration = results_df['duration_sec'].astype(float).mean()
        logger.info(f"Average duration for successful tasks: {avg_duration:.2f} seconds.")
        display(Markdown(f"### Average Duration: {avg_duration:.2f} seconds."))

        # Sort by 'qid'
        results_df.sort_values(by=['qid'], inplace=True)

        # remove duplicates based on 'qid' and keep the first occurrence
        results_df.drop_duplicates(subset=['qid'], keep='first', inplace=True)
        logger.info(f"Removed duplicates, remaining {len(results_df)} unique QIDs.")

        # select only the relevant columns
        results_df = results_df[['qid', 'CoT']]

        # Rename column
        results_df.rename(columns={'CoT': 'pred'}, inplace=True)

        new_name = COT_ONLY_OUTPUT_CSV.replace('.csv', "_") +  time.strftime("%Y%m%d_%H%M%S") + ".csv"
    
        # Save the cleaned DataFrame to a CSV file
        results_df.to_csv(new_name, index=False, encoding='utf-8')
        logger.info(f"Cleaned results saved to {new_name}.")
        display(Markdown(f"✅ Cleaned results saved to `{new_name}`."))
     
    except Exception as e:
        logger.error(f"Error processing results file: {e}", exc_info=True)
        display(Markdown(f"❌ Error processing results file: {e}."))
        return
  
results_cleanup_and_report()

### Delete original results file if needed

In [ ]:
os.remove(RESULTS_FINAL_DIR) # Remove the original results file
os.remove(RESULTS_FINAL_DIR.replace(".csv", "_failed.csv")) # Remove the failed results file